In [ ]:
#################################
# 네이버 블로그 크롤링
#################################
import requests
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd
import pprint
import re
import copy
import math

# 게시글 가져올 기간 설정(숫자형으로)
SEARCH_START_DATE = 20190101
SEARCH_END_DATE = 20191231

# 블로그 검색 URL(네이버)
BLOG_URL = "&".join(["https://search.naver.com/search.naver?",
            "where=post",
            "query={query_word}",
            "oquery={query_word}",
            "date_from="+str(SEARCH_START_DATE),
            "date_to="+str(SEARCH_END_DATE),
            "date_option=8",
            "st=date",
            "dup_remove=1",
            "srchby=all",
            "post_blogurl=blog.naver.com"])
    
# 검색제외 단어(+-) 밑에 코드에서 index 0번에 keyword 넣을거임
EXCEPT_WORD = ["맛집","펜션","숙소","호텔","근처","식당","카페","밥집","식사","게스트하우스","부동산","원룸","에어비앤비"]

# 관광지 목록 가져오기
FILE_PATH = "관광지_분류위도경도_0122.csv"
location_data = pd.read_csv(FILE_PATH, 
                       names=["분류", "장소명", "카카오맵_장소명", "검색_장소명", "위도", "경도", "주소"], 
                       encoding="euc-kr", header=0)

# 검색결과 목록에서 각 블로그글의 게시일자, 작성자ID를 가져온다.
def get_data(result_list, loc_category_name, location_name):
    post_info = [] # (작성자ID,게시일자)
    
    if len(result_list) > 0:
        for i in range(len(result_list)):
            post_one = result_list[i]

            post_date = post_one.find("dd", {"class" : "txt_inline"}).text.replace(".", "") # 게시일자(YYYYMMDD)
            post_id = post_one.find("a", {"class" : "url"}).attrs["href"].split("?")[0].split("/")[3] # 작성자ID
            
            post_info.append((loc_category_name, location_name, post_date, post_id))
        
    return post_info

# 추가되는 값이 있는 배열들의 값을 다른 배열에 저장한다.
def append_list(base_list, add_list):
    for one in add_list:
        base_list.append(one)

# 블로그 검색이 안되는 관광지명 별도 저장
no_result = []

# 브라우저 열기
driver = webdriver.Chrome("chromedriver.exe")

# 관광지명으로 네이버 블로그 크롤링
for i in range(len(location_data)):
    print("***현재까지 진행완료", i, " / ", len(location_data))
    
    post_info_list = [] # 관광지 1곳의 각 게시글에 대한 정보 배열 초기화
    location = location_data.iloc[i]
    category_name = location["분류"]
    
    # 관광지명으로 검색을 위한 URL 설정
    keyword = location["검색_장소명"].replace(" ", "") # 공백제거
    print("현재 검색어 :", keyword)
    
    query_word = copy.copy(EXCEPT_WORD)
    query_word.insert(0, "\""+keyword+"\"")
    query_word = "+-".join(query_word)
    url_params = {"query_word" : query_word}
    url = BLOG_URL.format(**url_params)
    
    search_page  = 1 # 검색 페이지 번호
    search_count = 0 # 총 검색건수
    search_over_1000 = False # 검색결과 1000건 이상
    
    # 1개의 관광지에 대하여 페이지만 이동하여 크롤링
    while True:
        # 페이지 지정 포함 URL
        page_url = url + "&start=" + str(search_page)

        # chrome driver load 후 검색
        driver.get(page_url)
        sleep(3)

        result_page_source = driver.page_source
        bsObj = BeautifulSoup(result_page_source, "lxml")
        
        search_count_area = bsObj.find("span", {"class" : "title_num"}) # 현재건수, 총건수 출력
        
        # 검색결과가 있는 경우
        if search_count_area != None and len(search_count_area) > 0:
            if search_page == 1:
                search_count = int(search_count_area.text.split("/")[1].replace(",","").replace("건", ""))
                print("search_count설정 :", search_count, "\t", category_name, keyword)
                
                # 1000건 이상인 경우는 나중에 따로 검색하기 위해 별도 저장
                if search_count >= 1000:
                    search_over_1000 = True
                    no_result.append((category_name, keyword))
                    break
            
            result_area = bsObj.find("ul", {"id" : "elThumbnailResultArea"})
        
            if result_area != None and len(result_area) > 0:
                blog_list = result_area.find_all("li", {"class" : "sh_blog_top"}, recursive = False)
                result = get_data(blog_list, category_name, keyword)

                if len(result) > 0:
                    append_list(post_info_list, result)

        # 검색결과가 없는 경우
        else:
            print("검색결과 없음->", category_name, keyword)
            no_result.append((category_name, keyword))
            break

        search_page += 10 # 다음 이동할 페이지(다음 페이지의 첫번째 index값)

        # 다음 이동할 페이지가 없으면 loop 종료
        if search_page > (math.trunc(search_count / 10) * 10 + 1):
            print("마지막 페이지 도달->", category_name, keyword)
            break
            
    if search_over_1000 :
        continue
            
    # 관광지 1개의 DataFrame 생성(게시일자, 게시자ID)
    if len(post_info_list) > 0:
        df = pd.DataFrame(post_info_list, columns=["분류", "검색_장소명", "게시일자", "게시자수"])
        df = df.drop_duplicates().groupby(["분류", "검색_장소명", "게시일자"]).count()
        print(df)

        # csv파일로 내보내기
        if len(df) > 0:
            df.to_csv("./naverblog_result/result.csv", 
                      mode="a",
                      header=False,
                      index=True,
                      encoding="euc-kr")
            
# 관광지 1개의 DataFrame 생성(게시일자, 게시자ID)
if len(no_result) > 0:
    no_result_df = pd.DataFrame(no_result, columns=["분류", "검색_장소명"])

    # csv파일로 내보내기
    if len(no_result_df) > 0:
        no_result_df.to_csv("./naverblog_result/no_result.csv", 
                  mode="a",
                  header=False,
                  index=True,
                  encoding="euc-kr")
    

***현재까지 진행완료 0  /  282
현재 검색어 : 가파도
search_count설정 : 2614 	 자연 가파도
***현재까지 진행완료 1  /  282
현재 검색어 : 갈마못
검색결과 없음-> 자연 갈마못
***현재까지 진행완료 2  /  282
현재 검색어 : 감낭오름
search_count설정 : 17 	 자연 감낭오름
마지막 페이지 도달-> 자연 감낭오름
                     게시자수
분류 검색_장소명 게시일자           
자연 감낭오름   20190131      1
          20190209      1
          20190223      1
          20190301      1
          20190323      1
          20190406      1
          20190608      1
          20190616      1
          20190808      1
          20190916      1
          20191008      1
          20191105      1
          20191109      1
          20191111      1
          20191112      1
          20191223      1
          20191225      1
***현재까지 진행완료 3  /  282
현재 검색어 : 갑선이오름
search_count설정 : 19 	 자연 갑선이오름
마지막 페이지 도달-> 자연 갑선이오름
                     게시자수
분류 검색_장소명 게시일자           
자연 갑선이오름  20190107      1
          20190130      1
          20190223      1
          20190318      1
          20190320      1
          20190327      1